# AutoML Image Instance Segmentation in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Image Instance Segmentation AutoML task.

**Motivations** - This notebook explains how to use Image Instance Segmentation AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential

from azure.ai.ml import MLClient, Input, command, Output
from azure.ai.ml.automl import (
    image_instance_segmentation,
    SearchSpace,
    InstanceSegmentationPrimaryMetrics,
)
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.sweep import BanditPolicy, Choice, Uniform

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

## 1.3 Download Data

Load the 'fridge items' dataset from a JSON file and MLTable definition.

In order to generate models for computer vision, you will need to bring in labeled image data as input for model training in the form of an Azure Machine Learning MLTable. 

In this notebook, we use a toy dataset called Fridge Objects, which consists of 134 images of 4 classes of beverage container {can, carton, milk bottle, water bottle} photos taken on different backgrounds.

All images in this notebook are hosted in [this repository](https://github.com/microsoft/computervision-recipes) and are made available under the [MIT license](https://github.com/microsoft/computervision-recipes/blob/master/LICENSE).

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

In [ ]:
import os
import urllib
from zipfile import ZipFile

# download data
download_url = "https://cvbp-secondary.z19.web.core.windows.net/datasets/object_detection/odFridgeObjectsMask.zip"
data_file = "./data/odFridgeObjectsMask.zip"
urllib.request.urlretrieve(download_url, filename=data_file)

# extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path="./data")
    print("done")
# delete zip file
os.remove(data_file)

This is a sample image from this dataset:

In [ ]:
from IPython.display import Image

sample_image = "./data/odFridgeObjectsMask/images/31.jpg"
Image(filename=sample_image)

### Upload the images to Datastore through an AML Data asset (URI Folder)

In order to use the data for training in Azure ML, we upload it to our default Azure Blob Storage of our  Azure ML Workspace.

Reference to URI FOLDER data asset example for further details: https://github.com/Azure/azureml-examples/blob/samuel100/data-samples/sdk/assets/data/data.ipynb

In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="./data/odFridgeObjectsMask",
    type=AssetTypes.URI_FOLDER,
    description="Fridge-items images instance segmentation",
    name="fridge-items-images-instance-segmentation",
)

uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

In [ ]:
# The jsonl_converter below relies on scikit-image and simplification.
# If you don't have them installed, install them before converting data by runing this cell.
%pip install "scikit-image==0.17.2" "simplification"

In [ ]:
from jsonl_converter import convert_mask_in_VOC_to_jsonl

data_path = "./data/odFridgeObjectsMask/"
convert_mask_in_VOC_to_jsonl(data_path, uri_folder_data_asset.path)

# 2. Basic pipeline job with Image Instance Segmentation task

## 2.1 Build pipeline

In [ ]:
# note that the used docker image doesn't suit for all size of gpu compute. Please use the following command to create gpu compute if experiment failed
# !az ml compute create -n gpu-cluster --type amlcompute --min-instances 0 --max-instances 4 --size Standard_NC12

In [ ]:
# Define pipeline
@pipeline(
    description="AutoML Image Instance Segmentation Pipeline",
)
def automl_image_instance_segmentation(
    image_instance_segmentation_train_data, image_instance_segmentation_validation_data
):
    # define the automl image_instance_segmentation task with automl function
    image_instance_segmentation_node = image_instance_segmentation(
        training_data=image_instance_segmentation_train_data,
        validation_data=image_instance_segmentation_validation_data,
        target_column_name="label",
        primary_metric=InstanceSegmentationPrimaryMetrics.MEAN_AVERAGE_PRECISION,
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes
        outputs={"best_model": Output(type="mlflow_model")},
    )
    image_instance_segmentation_node.set_limits(timeout_minutes=180)
    image_instance_segmentation_node.set_training_parameters(nms_iou_threshold=0.7)
    image_instance_segmentation_node.extend_search_space(
        [
            SearchSpace(
                model_name=Choice(["maskrcnn_resnet50_fpn"]),
                learning_rate=Uniform(0.0001, 0.001),
                optimizer=Choice(["sgd", "adam", "adamw"]),
                min_size=Choice([600, 800]),
            ),
        ]
    )

    image_instance_segmentation_node.set_sweep(
        max_trials=10,
        max_concurrent_trials=2,
        sampling_algorithm="Random",
        early_termination=BanditPolicy(
            evaluation_interval=2, slack_factor=0.2, delay_evaluation=6
        ),
    )

    # define command function for registering the model
    command_func = command(
        inputs=dict(
            model_input_path=Input(type="mlflow_model"),
            model_base_name="image_instance_segmentation_example_model",
        ),
        code="./register.py",
        command="python register.py "
        + "--model_input_path ${{inputs.model_input_path}} "
        + "--model_base_name ${{inputs.model_base_name}}",
        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1",
    )
    register_model = command_func(
        model_input_path=image_instance_segmentation_node.outputs.best_model
    )


data_folder = "./data"
pipeline = automl_image_instance_segmentation(
    image_instance_segmentation_train_data=Input(
        path=f"{data_folder}/training-mltable-folder/", type="mltable"
    ),
    image_instance_segmentation_validation_data=Input(
        path=f"{data_folder}/validation-mltable-folder/", type="mltable"
    ),
)

# set pipeline level compute
pipeline.settings.default_compute = "gpu-cluster"

# 2.2 Submit pipeline job

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)